# Brain Tumor Classification with CNN

This notebook demonstrates an end-to-end workflow for brain tumor classification using a CNN model, utilizing the existing project code structure. The dataset should be stored in your Google Drive.

## Overview

1. Mount Google Drive and set up the environment
2. Clone the repository (if needed)
3. Install dependencies
4. Set up paths to the dataset and results directories
5. Preprocess the data
6. Train the CNN model
7. Evaluate performance
8. Visualize results including Grad-CAM

## 1. Mount Google Drive and Setup Environment

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Verify TensorFlow and GPU availability
import tensorflow as tf
import platform

print('TensorFlow version:', tf.__version__)
print('Python version:', platform.python_version())
print('GPUs available:', tf.config.list_physical_devices('GPU'))

# Set seed for reproducibility
import numpy as np
import random
import os

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

## 2. Clone Repository (if needed)

If the repository is not already in your Colab environment, clone it here.

In [ ]:
# Check if repo exists, clone if it doesn't
import os

repo_name = "SE4050-Deep-Learning-Assignment"
if not os.path.exists(repo_name):
    !git clone https://github.com/IT22052124/SE4050-Deep-Learning-Assignment.git
    
%cd {repo_name}

## 3. Install Dependencies

In [ ]:
# Install required packages
!pip install -U pip
!pip install -r requirements.txt

## 4. Set Up Paths

Define paths to your dataset in Google Drive and where to save results.

In [ ]:
# Customize these paths according to your Google Drive structure
# Define the correct paths based on your folder structure
DRIVE_ROOT = '/content/drive/MyDrive'
BRAIN_TUMOR_DIR = DRIVE_ROOT + '/BrainTumor'
DATA_DIR = BRAIN_TUMOR_DIR + '/data'
RAW_DATA_DIR = DATA_DIR + '/archive'  # Contains yes/no folders with raw images
PROCESSED_DATA_DIR = DATA_DIR + '/processed'  # Where preprocessed data will be stored
RESULTS_DIR = BRAIN_TUMOR_DIR + '/Result/cnn'  # Where model and results will be saved

# Create directories if they don't exist
!mkdir -p {PROCESSED_DATA_DIR}
!mkdir -p {RESULTS_DIR}

print("Folder structure:")
print(f"- Brain Tumor Directory: {BRAIN_TUMOR_DIR}")
print(f"- Raw Data Directory: {RAW_DATA_DIR}")
print(f"- Processed Data Directory: {PROCESSED_DATA_DIR}")
print(f"- Results Directory: {RESULTS_DIR}")

## 5. Preprocess the Data

Run preprocessing script to prepare the dataset.

In [ ]:
# Let's verify the existence of raw data directories first
import os
from pathlib import Path
import cv2
import random
import shutil
from tqdm.notebook import tqdm

print("Checking raw data directories...")
print(f"Looking for raw data in: {RAW_DATA_DIR}")

# First verify the raw data exists and list what's available
if os.path.exists(RAW_DATA_DIR):
    dirs = [d for d in os.listdir(RAW_DATA_DIR) if os.path.isdir(os.path.join(RAW_DATA_DIR, d))]
    print(f"Found directories: {dirs}")
    
    # Check for yes/no directories specifically
    yes_dir = os.path.join(RAW_DATA_DIR, "yes")
    no_dir = os.path.join(RAW_DATA_DIR, "no")
    
    yes_exists = os.path.exists(yes_dir)
    no_exists = os.path.exists(no_dir)
    
    print(f"'yes' directory exists: {yes_exists}")
    print(f"'no' directory exists: {no_exists}")
    
    # Count files in each directory
    if yes_exists:
        yes_files = [f for f in os.listdir(yes_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        print(f"Found {len(yes_files)} image files in 'yes' directory")
        if yes_files:
            print(f"Sample files: {yes_files[:3]}")
    
    if no_exists:
        no_files = [f for f in os.listdir(no_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        print(f"Found {len(no_files)} image files in 'no' directory")
        if no_files:
            print(f"Sample files: {no_files[:3]}")
            
    # If the directories don't exist, search for alternatives
    if not (yes_exists and no_exists):
        print("\nSearching for alternative image directories...")
        all_subdirs = []
        for root, dirs, _ in os.walk(RAW_DATA_DIR):
            for d in dirs:
                subdir_path = os.path.join(root, d)
                img_count = len([f for f in os.listdir(subdir_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
                if img_count > 0:
                    all_subdirs.append((subdir_path, img_count))
        
        if all_subdirs:
            print("Found alternative directories with images:")
            for path, count in all_subdirs:
                print(f"  - {path}: {count} images")
        else:
            print("No alternative directories with images found.")
else:
    print(f"❌ Error: Raw data directory {RAW_DATA_DIR} does not exist!")
    print("Please make sure your Google Drive contains the correct folder structure.")
    
# Now let's define our preprocessing functions
# Constants
RANDOM_SEED = 42
IMG_SIZE = (224, 224)
VAL_SPLIT = 0.15
TEST_SPLIT = 0.15

def create_dirs(base_path, classes):
    """Create directory structure for processed data"""
    for split in ["train", "val", "test"]:
        for cls in classes:
            os.makedirs(os.path.join(base_path, split, cls), exist_ok=True)
            
def split_and_copy(source_root, dest_root, classes):
    """Split and copy files into train/val/test directories"""
    random.seed(RANDOM_SEED)
    create_dirs(dest_root, classes)
    
    # Track total files processed
    total_processed = 0
    
    for cls in classes:
        cls_dir = os.path.join(source_root, cls)
        if not os.path.exists(cls_dir):
            print(f"Warning: Class directory {cls_dir} not found.")
            continue
            
        # Get all image files
        imgs = []
        for ext in ['.jpg', '.jpeg', '.png']:
            imgs.extend([os.path.join(cls_dir, f) for f in os.listdir(cls_dir) 
                         if f.lower().endswith(ext)])
            
        if not imgs:
            print(f"No images found in {cls_dir}")
            continue
            
        print(f"Found {len(imgs)} images in {cls_dir}")
        random.shuffle(imgs)
        n = len(imgs)
        n_val, n_test = int(n * VAL_SPLIT), int(n * TEST_SPLIT)
        n_train = n - n_val - n_test
        
        splits = {
            "train": imgs[:n_train],
            "val": imgs[n_train:n_train+n_val],
            "test": imgs[n_train+n_val:]
        }
        
        for split, files in splits.items():
            print(f"Processing {cls} -> {split}: {len(files)} images")
            for img_path in tqdm(files):
                try:
                    # Read and resize image
                    img = cv2.imread(img_path)
                    if img is None:
                        print(f"Warning: Could not read {img_path}")
                        continue
                    
                    # Resize image
                    resized = cv2.resize(img, IMG_SIZE)
                    
                    # Save to destination
                    out_path = os.path.join(dest_root, split, cls, os.path.basename(img_path))
                    cv2.imwrite(out_path, resized)
                    total_processed += 1
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")
    
    return total_processed

print("\nStarting preprocessing...")
print(f"Reading raw images from: {RAW_DATA_DIR}")
print(f"Saving processed images to: {PROCESSED_DATA_DIR}")

# Process the data
if os.path.exists(RAW_DATA_DIR) and (os.path.exists(os.path.join(RAW_DATA_DIR, "yes")) or 
                                     os.path.exists(os.path.join(RAW_DATA_DIR, "no"))):
    total_files = split_and_copy(RAW_DATA_DIR, PROCESSED_DATA_DIR, ["yes", "no"])
    print(f"✅ Preprocessing completed successfully! Processed {total_files} images.")
else:
    print("⚠️ Could not find expected yes/no folders in the raw data directory.")
    print("Please make sure your Google Drive contains the correct folder structure,")
    print("or manually upload the brain tumor dataset with 'yes' and 'no' subfolders.")
    
# Count files in each split to verify
print("\nVerifying processed data:")
total_processed_files = 0
for split in ["train", "val", "test"]:
    split_dir = os.path.join(PROCESSED_DATA_DIR, split)
    if os.path.exists(split_dir):
        yes_path = os.path.join(split_dir, "yes")
        no_path = os.path.join(split_dir, "no")
        
        yes_count = len(os.listdir(yes_path)) if os.path.exists(yes_path) else 0
        no_count = len(os.listdir(no_path)) if os.path.exists(no_path) else 0
        split_total = yes_count + no_count
        total_processed_files += split_total
        
        print(f"{split.upper()}: yes={yes_count}, no={no_count}, total={split_total}")

print(f"Total processed images: {total_processed_files}")

# If no files were processed, show a warning
if total_processed_files == 0:
    print("\n⚠️ WARNING: No images were processed. Training will likely fail.")
    print("Please check your Google Drive folder structure and raw data.")

## 6. Train the CNN Model

Run the training script to train the CNN model on the preprocessed data.

In [ ]:
# First, let's verify that we have processed data to train on
import os

# Check if processed data exists with the right structure
train_dir = os.path.join(PROCESSED_DATA_DIR, "train")
val_dir = os.path.join(PROCESSED_DATA_DIR, "val")
test_dir = os.path.join(PROCESSED_DATA_DIR, "test")

processed_data_exists = (os.path.exists(train_dir) and 
                         os.path.exists(val_dir) and 
                         os.path.exists(test_dir))

# Check if we have class folders in the train directory
class_folders_exist = False
if processed_data_exists:
    train_yes = os.path.join(train_dir, "yes")
    train_no = os.path.join(train_dir, "no")
    
    class_folders_exist = (os.path.exists(train_yes) and 
                           os.path.exists(train_no))
    
    if class_folders_exist:
        yes_count = len(os.listdir(train_yes))
        no_count = len(os.listdir(train_no))
        
        print(f"Train directory has {yes_count} 'yes' images and {no_count} 'no' images")
        
        if yes_count == 0 or no_count == 0:
            class_folders_exist = False
            print("⚠️ One of the class folders is empty!")

# Create a simple modified script to handle the train/val/test structure
# We'll create this script on the fly since we don't want to modify your existing code
%%writefile /content/modified_train.py
import os
import sys
import json
import tensorflow as tf
from tensorflow.keras import layers, models
import argparse
import matplotlib.pyplot as plt

# Add the project path to sys.path
sys.path.append('/content/SE4050-Deep-Learning-Assignment')

# Import necessary modules from your project
from src.models.cnn.build_cnn import build_cnn_model
from src.common.preprocessing import get_augmentation_pipeline

def parse_args():
    parser = argparse.ArgumentParser(description="Train CNN for Brain Tumor classification")
    parser.add_argument("--data_dir", type=str, required=True, help="Folder containing train/val/test splits")
    parser.add_argument("--results_dir", type=str, required=True, help="Folder to write checkpoints and plots")
    parser.add_argument("--batch_size", type=int, default=32)
    parser.add_argument("--epochs", type=int, default=30)
    parser.add_argument("--img_size", type=int, nargs=2, default=(224, 224))
    return parser.parse_args()

def create_dataset(data_path, img_size=(224, 224), batch_size=32, is_training=False):
    """Create a TF dataset from a directory with class subdirectories"""
    if is_training:
        datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255,
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            fill_mode='nearest'
        )
    else:
        datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
        
    return datagen.flow_from_directory(
        data_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        shuffle=is_training
    )

def main():
    args = parse_args()
    os.makedirs(args.results_dir, exist_ok=True)
    
    # Create datasets from the train/val/test directories
    train_dir = os.path.join(args.data_dir, 'train')
    val_dir = os.path.join(args.data_dir, 'val')
    test_dir = os.path.join(args.data_dir, 'test')
    
    # Check if directories exist
    if not os.path.exists(train_dir) or not os.path.exists(val_dir) or not os.path.exists(test_dir):
        print(f"Error: One or more data directories not found in {args.data_dir}")
        sys.exit(1)
        
    print(f"Loading data from train: {train_dir}, val: {val_dir}, test: {test_dir}")
    
    train_generator = create_dataset(train_dir, img_size=tuple(args.img_size), 
                                     batch_size=args.batch_size, is_training=True)
    val_generator = create_dataset(val_dir, img_size=tuple(args.img_size), 
                                   batch_size=args.batch_size, is_training=False)
    test_generator = create_dataset(test_dir, img_size=tuple(args.img_size), 
                                    batch_size=args.batch_size, is_training=False)
    
    class_names = list(train_generator.class_indices.keys())
    print(f"Class names: {class_names}")
    
    # Save class names for evaluation
    with open(os.path.join(args.results_dir, "class_names.json"), "w") as f:
        json.dump(class_names, f, indent=2)
    
    # Calculate class weights to handle imbalance
    from sklearn.utils.class_weight import compute_class_weight
    import numpy as np
    
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(train_generator.classes),
        y=train_generator.classes
    )
    
    class_weight_dict = {i: w for i, w in enumerate(class_weights)}
    print(f"Class weights: {class_weight_dict}")
    
    # Build model
    model = build_cnn_model((*args.img_size, 3))
    model.summary()
    
    # Learning rate schedule: cosine decay
    initial_lr = 1e-3
    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=initial_lr, decay_steps=args.epochs * 100
    )
    
    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    # Callbacks
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(args.results_dir, "best_model.keras"),
            monitor="val_loss",
            save_best_only=True,
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=5, restore_best_weights=True
        ),
    ]
    
    # Train
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=args.epochs,
        callbacks=callbacks,
        class_weight=class_weight_dict
    )
    
    # Plot and save training history
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training')
    plt.plot(history.history['val_accuracy'], label='Validation')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(args.results_dir, "history.png"))
    
    # Save metrics
    metrics = {
        "val_acc": float(history.history["val_accuracy"][-1]),
        "train_acc": float(history.history["accuracy"][-1])
    }
    
    with open(os.path.join(args.results_dir, "metrics.json"), "w") as f:
        json.dump(metrics, f, indent=4)
        
    print("✅ Training complete. Best model saved to:", args.results_dir)

if __name__ == "__main__":
    main()


# Choose the right directory for training
if processed_data_exists and class_folders_exist:
    # If processed data with correct structure exists, use it
    TRAIN_DATA_DIR = PROCESSED_DATA_DIR
    print(f"Using processed data for training: {TRAIN_DATA_DIR}")
    print("Using our modified script that handles the train/val/test structure")
elif os.path.exists(os.path.join(RAW_DATA_DIR, "yes")) and os.path.exists(os.path.join(RAW_DATA_DIR, "no")):
    # If raw data exists with yes/no folders, use the original script
    TRAIN_DATA_DIR = RAW_DATA_DIR
    print(f"Using raw data for training: {TRAIN_DATA_DIR}")
    print("Using the original training script")
else:
    # Fall back to the original DATA_DIR (parent of archive)
    TRAIN_DATA_DIR = DATA_DIR
    print(f"⚠️ Could not find properly structured data. Trying: {TRAIN_DATA_DIR}")
    print("Training may fail if the correct data structure is not found.")

# Train the model
print("\nTraining the CNN model...")
print(f"Using data from: {TRAIN_DATA_DIR}")
print(f"Saving results to: {RESULTS_DIR}")

# Check if we're using the processed data with train/val/test structure
if processed_data_exists and class_folders_exist:
    # Use our modified script for the train/val/test structure
    !python /content/modified_train.py \
        --data_dir {PROCESSED_DATA_DIR} \
        --results_dir {RESULTS_DIR} \
        --epochs 30 \
        --batch_size 32 \
        --img_size 224 224
else:
    # Use the original script which expects yes/no folders directly
    !python -m src.models.cnn.train_cnn \
        --data_dir {TRAIN_DATA_DIR} \
        --results_dir {RESULTS_DIR} \
        --epochs 30 \
        --batch_size 32 \
        --img_size 224 224

## 7. Evaluate the Model

Run the evaluation script to assess model performance on the test set. The script has been enhanced to automatically detect whether it should use the train/val/test folder structure or the original structure with class folders directly under the data directory.

In [ ]:
# Evaluate the model on the test set
print("Evaluating the CNN model...")

# Choose the right directory for evaluation
if processed_data_exists and class_folders_exist:
    # If processed data with correct structure exists, use it
    EVAL_DATA_DIR = PROCESSED_DATA_DIR
    print(f"Using processed data for evaluation: {EVAL_DATA_DIR}")
else:
    # Fall back to the training data directory
    EVAL_DATA_DIR = TRAIN_DATA_DIR
    print(f"Using training data directory for evaluation: {EVAL_DATA_DIR}")

print(f"Results will be saved to: {RESULTS_DIR}")

# Use the enhanced evaluate_cnn script
# The script now automatically detects whether to use the train/val/test structure
!python -m src.models.cnn.evaluate_cnn \
    --data_dir {EVAL_DATA_DIR} \
    --results_dir {RESULTS_DIR} \
    --batch_size 32 \
    --img_size 224 224 \
    --use_processed {1 if processed_data_exists and class_folders_exist else 0}

## 8. Display Results

Show evaluation metrics, plots, and visualizations stored in the results directory.

In [ ]:
# Display training history plot
import matplotlib.pyplot as plt
from PIL import Image

try:
    history_img = Image.open(f"{RESULTS_DIR}/history.png")
    plt.figure(figsize=(10, 6))
    plt.imshow(history_img)
    plt.axis('off')
    plt.title('Training History')
    plt.show()
except Exception as e:
    print(f"Error displaying training history: {e}")

In [ ]:
# Display confusion matrix
try:
    cm_img = Image.open(f"{RESULTS_DIR}/confusion_matrix.png")
    plt.figure(figsize=(8, 8))
    plt.imshow(cm_img)
    plt.axis('off')
    plt.title('Confusion Matrix')
    plt.show()
except Exception as e:
    print(f"Error displaying confusion matrix: {e}")

In [ ]:
# Display ROC curve
try:
    roc_img = Image.open(f"{RESULTS_DIR}/roc_curve.png")
    plt.figure(figsize=(8, 8))
    plt.imshow(roc_img)
    plt.axis('off')
    plt.title('ROC Curve')
    plt.show()
except Exception as e:
    print(f"Error displaying ROC curve: {e}")

In [ ]:
# Display Precision-Recall curve
try:
    pr_img = Image.open(f"{RESULTS_DIR}/precision_recall_curve.png")
    plt.figure(figsize=(8, 8))
    plt.imshow(pr_img)
    plt.axis('off')
    plt.title('Precision-Recall Curve')
    plt.show()
except Exception as e:
    print(f"Error displaying Precision-Recall curve: {e}")

In [ ]:
# Display Grad-CAM visualizations
import glob

gradcam_images = glob.glob(f"{RESULTS_DIR}/gradcam/*.png")
if gradcam_images:
    plt.figure(figsize=(15, 10))
    for i, img_path in enumerate(gradcam_images[:5]):
        plt.subplot(1, min(5, len(gradcam_images)), i+1)
        img = Image.open(img_path)
        plt.imshow(img)
        plt.axis('off')
    plt.suptitle('Grad-CAM Visualizations')
    plt.tight_layout()
    plt.show()
else:
    print("No Grad-CAM visualizations found.")

In [ ]:
# Display classification report
try:
    with open(f"{RESULTS_DIR}/classification_report.txt", 'r') as f:
        report = f.read()
    print("Classification Report:")
    print(report)
except Exception as e:
    print(f"Error reading classification report: {e}")

In [ ]:
# Display metrics
import json

try:
    with open(f"{RESULTS_DIR}/metrics.json", 'r') as f:
        metrics = json.load(f)
    print("Model Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")
except Exception as e:
    print(f"Error reading metrics: {e}")

## 9. Make Predictions with the Model

Load the best model and make predictions on sample images.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import glob
import json

# Load the best model
try:
    # Try to load the Keras model first
    model_path = f"{RESULTS_DIR}/best_model.keras"
    if not os.path.exists(model_path):
        # Fallback to H5 format
        model_path = f"{RESULTS_DIR}/best_model.h5"
    
    model = tf.keras.models.load_model(model_path, compile=False)
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    print(f"Successfully loaded model from {model_path}")
    
    # Load class names
    try:
        with open(f"{RESULTS_DIR}/class_names.json", 'r') as f:
            class_names = json.load(f)
    except:
        class_names = ["no", "yes"]
    
    print(f"Classes: {class_names}")
except Exception as e:
    print(f"Error loading model: {e}")

In [ ]:
# Function to make predictions on sample images
def predict_and_display(image_path):
    # Load and preprocess image
    img = tf.io.read_file(image_path)
    img = tf.io.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, (224, 224))
    img_display = img.numpy().astype(np.uint8)
    img = tf.cast(img, tf.float32) / 255.0
    img = tf.expand_dims(img, axis=0)
    
    # Make prediction
    pred = model.predict(img, verbose=0)[0][0]
    predicted_class = class_names[1] if pred > 0.5 else class_names[0]
    confidence = pred if pred > 0.5 else 1 - pred
    
    # Display results
    plt.figure(figsize=(6, 6))
    plt.imshow(img_display)
    plt.title(f"Prediction: {predicted_class} ({confidence:.2f})")
    plt.axis('off')
    plt.show()
    
    return predicted_class, confidence

# Find some sample images to predict - use the processed test images
test_yes_dir = os.path.join(PROCESSED_DATA_DIR, "test", "yes") 
test_no_dir = os.path.join(PROCESSED_DATA_DIR, "test", "no")

yes_samples = []
no_samples = []

# Try to get processed test images first
if os.path.exists(test_yes_dir):
    yes_files = os.listdir(test_yes_dir)
    yes_samples = [os.path.join(test_yes_dir, f) for f in yes_files[:2]] if yes_files else []
    
if os.path.exists(test_no_dir):
    no_files = os.listdir(test_no_dir)
    no_samples = [os.path.join(test_no_dir, f) for f in no_files[:2]] if no_files else []

# If we couldn't find processed test images, try the raw images
if not yes_samples and os.path.exists(os.path.join(RAW_DATA_DIR, "yes")):
    yes_files = os.listdir(os.path.join(RAW_DATA_DIR, "yes"))
    yes_samples = [os.path.join(RAW_DATA_DIR, "yes", f) for f in yes_files[:2]] if yes_files else []
    
if not no_samples and os.path.exists(os.path.join(RAW_DATA_DIR, "no")):
    no_files = os.listdir(os.path.join(RAW_DATA_DIR, "no"))
    no_samples = [os.path.join(RAW_DATA_DIR, "no", f) for f in no_files[:2]] if no_files else []

sample_images = yes_samples + no_samples

if sample_images:
    print(f"Making predictions on {len(sample_images)} sample images")
    for img_path in sample_images:
        print(f"\nImage: {os.path.basename(img_path)}")
        true_class = "yes" if "yes" in img_path else "no"
        pred_class, conf = predict_and_display(img_path)
        print(f"True class: {true_class}")
        print(f"Predicted: {pred_class} with {conf:.2f} confidence")
else:
    print("No sample images found for prediction.")

## 10. Conclusion

This notebook has demonstrated an end-to-end workflow for brain tumor classification using a CNN model:

1. We set up the environment and mounted Google Drive
2. We prepared the dataset by:
   - Reading raw images from `/content/drive/MyDrive/BrainTumor/data/archive`
   - Preprocessing them (resize to 224x224)
   - Splitting into train/val/test sets (70/15/15%)
   - Saving to `/content/drive/MyDrive/BrainTumor/data/processed`
3. We trained a CNN model using the processed images
4. We evaluated the model and saved results to `/content/drive/MyDrive/BrainTumor/Result/cnn`
5. We visualized the results and made predictions on sample images

The data flow in this notebook follows your folder structure:
- Raw data → Preprocessing → Processed data → Training → Results

For further improvements, you could:
- Try different CNN architectures like ResNet50 or EfficientNet
- Experiment with different preprocessing techniques
- Apply more advanced data augmentation
- Adjust hyperparameters like learning rate, batch size, etc.